In [2]:
import geopandas as gpd
import numpy as np

In [3]:
# load shapefile
boroughs = gpd.read_file("shapefiles/Boroughs2020/")
boroughs['log_pop'] = np.log(boroughs.TOTALPOP)

In [4]:
boroughs.explore("log_pop", legend = True)